In [115]:
# import required packages
import pandas as pd
import requests
import numpy as np
import re

#from google.colab import files  # for downloading files
import bs4
from bs4 import BeautifulSoup

# suppress warnings
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# import advanced scraping packages
import requests, lxml.html

In [3]:
team_abb = pd.read_csv("/Users/ryancook/Desktop/Syracuse-Analytics-Blitz/data/nfl_teams.csv")

In [23]:
years_num = list(range(2014,2021))
years = [ str(i) for i in years_num ]

pos = ['running-back', 'wide-receiver', 'quarterback', 'fullback', 'tight-end']


urls = []
for year in range(len(years)):
    for p in range(len(pos)):
        url = 'https://www.spotrac.com/nfl/rankings/' + years[year] + '/cap-hit/' + pos[p]
        urls.append(url)


# flip order of urls for most recent years first
urls = urls[::-1] 

['https://www.spotrac.com/nfl/rankings/2014/cap-hit/running-back',
 'https://www.spotrac.com/nfl/rankings/2014/cap-hit/wide-receiver',
 'https://www.spotrac.com/nfl/rankings/2014/cap-hit/quarterback',
 'https://www.spotrac.com/nfl/rankings/2014/cap-hit/fullback',
 'https://www.spotrac.com/nfl/rankings/2014/cap-hit/tight-end',
 'https://www.spotrac.com/nfl/rankings/2015/cap-hit/running-back',
 'https://www.spotrac.com/nfl/rankings/2015/cap-hit/wide-receiver',
 'https://www.spotrac.com/nfl/rankings/2015/cap-hit/quarterback',
 'https://www.spotrac.com/nfl/rankings/2015/cap-hit/fullback',
 'https://www.spotrac.com/nfl/rankings/2015/cap-hit/tight-end',
 'https://www.spotrac.com/nfl/rankings/2016/cap-hit/running-back',
 'https://www.spotrac.com/nfl/rankings/2016/cap-hit/wide-receiver',
 'https://www.spotrac.com/nfl/rankings/2016/cap-hit/quarterback',
 'https://www.spotrac.com/nfl/rankings/2016/cap-hit/fullback',
 'https://www.spotrac.com/nfl/rankings/2016/cap-hit/tight-end',
 'https://www.sp

In [172]:
cap_stats = []
for i in range(len(urls)): 
    with requests.session() as s:

        r = s.post(urls[i], data={'ajax': True, 'mobile': False})

        # Get page content, find first table, and save to df
        soup = BeautifulSoup(r.content, 'lxml')
        table = soup.find_all('table')[0]
        df_list = pd.read_html(str(table))
        df = df_list[0]

        # fix rank values and col name
        df = df.rename(columns={'Unnamed: 0':'rank','POS':'pos'})
        df['rank'] = df['rank'].fillna(method='ffill').astype('int')

        player_df = df['Player'].str.split('(\\s[A-Z]{2}$|\\s[A-Z]{3}$)', expand=True)
        player_df.columns = ['player', 'team', 'drop']
        player_df = player_df.drop(columns='drop')

        df = pd.concat([df, player_df], axis=1).drop(columns='Player')
        df = df[['rank', 'player', 'team', 'cap hit', 'pos']]

        df['year'] = re.findall('[0-9]{4}', urls[i])[0]

        df['cap hit'] = df['cap hit'].str.replace('$', '').str.replace(',', '')


        cap_stats.append(df)
        
        print('-- Added cap stats for {0}s in {1} ({2} rows)' 
              .format(df['pos'][0], df['year'][0], cap_stats[i].shape[0]))
    
    

-- Added cap stats for TEs in 2020 (141 rows)
-- Added cap stats for FBs in 2020 (20 rows)
-- Added cap stats for QBs in 2020 (91 rows)
-- Added cap stats for WRs in 2020 (232 rows)
-- Added cap stats for RBs in 2020 (151 rows)
-- Added cap stats for TEs in 2019 (142 rows)
-- Added cap stats for FBs in 2019 (25 rows)
-- Added cap stats for QBs in 2019 (94 rows)
-- Added cap stats for WRs in 2019 (250 rows)
-- Added cap stats for RBs in 2019 (140 rows)
-- Added cap stats for TEs in 2018 (139 rows)
-- Added cap stats for FBs in 2018 (22 rows)
-- Added cap stats for QBs in 2018 (90 rows)
-- Added cap stats for WRs in 2018 (250 rows)
-- Added cap stats for RBs in 2018 (148 rows)


IndexError: list index out of range

In [169]:
cap_df = pd.concat(cap_stats)

In [170]:
cap_df.to_csv('/Users/ryancook/Downloads/spotrac_cap_hit.csv')